In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

  Using cached qiskit-1.2.4-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached stevedore-5.4.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached symengine-0.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached pbr-6.1.1-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached qiskit-1.2.4-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
Using cached dill-0.3.9-py3-none-any.whl (119 kB)
Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
Using cached scipy-1.15.2-

In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol with an attacker, to demonstrate that the attacker can be detected.

In [8]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
from random import randint
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.

root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 


Z = [[1,0], [0,1]]
X = [[1/root2, 1/root2], [1/root2, -1/root2]]
W = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]
V = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]

A = [1,1,1,1]
B = [1,1,1,1]

A[1] = X
A[2] = W
A[3] = Z
B[1] = W
B[2] = Z
B[3] = V

def entangle():
    circuit = QuantumCircuit(2, 2)
    circuit.h(0)
    circuit.z(0)
    circuit.cx(0, 1)
    circuit.x(0)

    # attack:

    circuit.measure(0,0)
    circuit.measure(1,1)
    return circuit

def measure_qubits(a, b):
    circuit = entangle()
    circuit.unitary(A[a], 0)
    circuit.unitary(B[b], 1)
    circuit.measure(0, 0)
    circuit.measure(1, 1)
    return circuit

def run_simulator(circuit):
    backend = BasicSimulator()
    compiled = transpile(circuit, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    return list(result_sim.get_counts().keys())[0]

circuit = measure_qubits(2, 3)

def random_third():
    circuit = QuantumCircuit(2)
    circuit.initialize([1 / math.sqrt(3), math.sqrt(2) / math.sqrt(3)], 0)
    circuit.initialize([1 / math.sqrt(2), 1 / math.sqrt(2)], 1)
    circuit.measure_all()
    backend = BasicSimulator()
    qc_compiled = transpile(circuit, backend)
    result = next(iter(backend.run(qc_compiled, shots=1).result().get_counts()))
    return 1 if result[1] == '0' else 2 if result[0] == '0' else 3

def single_run():
    a = random_third()
    b = random_third()
    measured = run_simulator(measure_qubits(a, b))
    return (a, b, measured)

key = ""
N = 100

bases = []
for i in range(4):
    row = []
    for j in range(4):
        row.append({"total_value":0, "count":0})
    bases.append(row)
bases[0] = []

def calc_value(m):
    if m[0] == m[1]:
        return 1
    else:
        return -1
            
for i in range(round((9*N)/2)):
    (a, b, m) = single_run()
    bases[a][b]["count"] += 1
    bases[a][b]["total_value"] += calc_value(m) 
    if (a,b) == (2,1) or (a,b) == (3,2):
        key += m[0]

s = 0
s += bases[1][1]["total_value"] / bases[1][1]["count"]
s -= bases[1][3]["total_value"] / bases[1][3]["count"]
s += bases[3][1]["total_value"] / bases[3][1]["count"]
s += bases[3][3]["total_value"] / bases[3][3]["count"]



print("key:", key)
print("length of key: ", len(key), "(N =", str(N) +")" )
print("S =", abs(s))
if abs(s) < 2.5:
    print("S value low. Interference likely")

key: 010000001110110010110110011110110001011010001100110001000011011111101100111010110110011000011110111011011001100111
length of key:  114 (N = 100)
S = 1.51146166574738
S value low. Interference likely
